In [ ]:
"""Program to calculate Bray-Curtis similarity values between two datasets.  
"""
numprocess = 410
import math
from google.colab import drive
drive.mount('/content/gdrive')

#1. THE OUTFILE NAME. IF A MG IS TO LEFT LEFT OUT,INSERT IT IN PLACE OF THE "0":
outname = '/content/gdrive/Shared drives/4.Venetian Canals/Joined MG Files/VC.Plus.3.Joined.BC.Matrix.csv'

#2. SPECIFY ALL THE INPUT METAGENOME FILES: 
VCfiles = ['/content/gdrive/Shared drives/4.Venetian Canals/Joined MG Files/Genus.Site1.Loop1.TRANSPOSED.csv',
         '/content/gdrive/Shared drives/4.Venetian Canals/Joined MG Files/Genus.Site2.Loop3.TRANSPOSED.csv',
         '/content/gdrive/Shared drives/4.Venetian Canals/Joined MG Files/Genus.Site3.Loop2.TRANSPOSED.csv',
         '/content/gdrive/Shared drives/4.Venetian Canals/Joined MG Files/Genus.Site4.Loop1.TRANSPOSED.csv',
         '/content/gdrive/Shared drives/4.Venetian Canals/Joined MG Files/Genus.Site5.Loop3.TRANSPOSED.csv',
         '/content/gdrive/Shared drives/4.Venetian Canals/Joined MG Files/Genus.Site6.Loop2.TRANSPOSED.csv']

altri =  ['/content/gdrive/Shared drives/4.Venetian Canals/Joined MG Files/A.K.Pri.4663797.3_organism_genus_hits.csv', #PRI
          '/content/gdrive/Shared drives/4.Venetian Canals/Joined MG Files/A.Z.USA.4.14_organism_genus_hits.csv.xls', #ASB
          '/content/gdrive/Shared drives/4.Venetian Canals/Joined MG Files/A.Z.USA.1.23_organism_genus_hits.csv.xls'] # ANR

files = VCfiles  + altri 
nummetas =  len(files)
dataset = str(nummetas)

mglabel =['VC1', 'VC2', 'VC3', 'VC4', 'VC5', 'VC6', 'PRI', 'ASB', 'ANR']
print(mglabel)

stringmetas =[]
for x in range(nummetas):   # for reading files and converting each taxon to a list of [taxon, number]
  with open(files[x], 'r') as f:
    data = f.read()
    #print(repr(data)[:100])
    stringmetas.append(str(data))
  f.close()
print('Number of metagenomes =', len(stringmetas))
print( 'column labels =', mglabel)

cleanmetas =[]            # gets rid of unwanted text, converts number from string to integer
for x in range(nummetas):
  datastring = stringmetas[x].replace('unclassified (derived from ', '').replace(')', '').replace('Candidatus ', '')
  metalines = datastring.split('\n')
  metalines.pop()
  integermeta =[]
  for y in range(numprocess):  #len(metalines)):
    row = metalines[y].split(',')
    integermeta.append([row[0], int(row[1]), 0.0])
  cleanmetas.append(integermeta)

totalslist =[]             # counts the total number of taxa in each metagenome
for x in range(nummetas):
  count =0
  for y in range(numprocess): #len(cleanmetas[x])):
    count += cleanmetas[x][y][1]
  totalslist.append(count)
print('Total counts for each metagenone: ', totalslist)

for x in range(nummetas):
  for y in range(len(cleanmetas[x])):
     cleanmetas[x][y][2] = cleanmetas[x][y][1] / totalslist[x]

### need to make union of all genera
taxonset = set()
for x in range(nummetas):
  for y in range(numprocess):
    if cleanmetas[x][y][0] not in taxonset:
      taxonset.add(cleanmetas[x][y][0])

taxonlist = list(taxonset)
print('Total taxa in the union =', len(taxonlist))

### need to assign '0' values to missing taxa in each metagenome
for x in range(len(cleanmetas)):
  genera = set()
  for y in range(len(cleanmetas[x])):
    genera.add(cleanmetas[x][y][0])
  for z in range(len(taxonlist)):
    if taxonlist[z] not in genera:
      cleanmetas[x].append([taxonlist[z], 0, 0.0])

for x in range(len(cleanmetas)):
  cleanmetas[x].sort(key = lambda x: x[0])
  cleanmetas[x].reverse()

# make the BC matrix as a table
table = []
for x in range(len(cleanmetas)):                   # go through each meta (len = 7)
  row =[]                                         # list of BC values 
  for y in range(len(cleanmetas)):                  
    bcsum =0
    for z in range(len(taxonlist)):    
      bcsum += abs( cleanmetas[x][z][2] - cleanmetas[y][z][2] )
    row.append(bcsum /2 )    
  table.append(row)
  #print(row)

g = open(outname, 'w')
for x in range(len(table)):
  outline = mglabel[x] +','
  for y in range(len(table[x]) -1):
    outline += str(table[x][y]) + ','
  outline += str(table[x][len(table[x]) -1]) + '\n'
  print(outline)
  g.write(outline)
g.close()  

print("Done!")


Mounted at /content/gdrive
['VC1', 'VC2', 'VC3', 'VC4', 'VC5', 'VC6', 'PRI', 'ASB', 'ANR']
Number of metagenomes = 9
column labels = ['VC1', 'VC2', 'VC3', 'VC4', 'VC5', 'VC6', 'PRI', 'ASB', 'ANR']
Total counts for each metagenone:  [36330, 17457, 20288, 11420, 17599, 16848, 1412032, 1618875, 1092423]
Total taxa in the union = 658
VC1,0.0,0.24821107918018895,0.2176931623107842,0.21121799924124207,0.3836159600252543,0.24746472663139324,0.7183052759411569,0.7389032022561566,0.5835116294447432

VC2,0.24821107918018895,0.0,0.10650551969155769,0.15469664415350526,0.204982796640189,0.12469311039726924,0.734776445727299,0.6995839546389176,0.6222841990473217

VC3,0.2176931623107842,0.10650551969155769,0.0,0.15118388031954566,0.23405265724787594,0.12436446977390715,0.7326208798263529,0.7027981560319443,0.6160789929282345

VC4,0.21121799924124207,0.15469664415350526,0.15118388031954566,0.0,0.28192408938216845,0.14532359383497725,0.7342546559871184,0.7139186877224557,0.6026341340455691

VC5,0.3836